In [ ]:
# Environment

In [ ]:
## Librairies

In [62]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import operator
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm

from shutil import copyfile
import shutil
import pickle

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.callbacks import ReduceLROnPlateau

## Data Loading

In [71]:
project_path = './../'
seatguru_path = project_path + 'Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/'
stats_path = project_path + 'ImagesStats/'
split_path = project_path + 'G7_SEATGURU/View/'
new_paths = [split_path + 'data_test/', split_path + 'data_train/']


# Read SEATGURU annotated CSV
df_seat_annot = pd.read_csv(stats_path + 'g7_SEATGURU_annotate.csv', sep=';')

model_name = 'View_F'

# Classes to predict
views = ['Ext', 'Int', 'Meal' ,'Ext_Int']
nb_types = len(views)

# Images parameters
size = (224, 224)
greys = False

split_limit = 0.7
s = 8

In [72]:
## Fonctions

In [73]:
%run g7_functions_for_models_V2.ipynb

In [74]:
split_train_test_seatguru_view(new_paths, seatguru_path, views, df_seat_annot)

In [75]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

In [76]:
train_generator = train_datagen.flow_from_directory(new_paths[0],
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1746 images belonging to 4 classes.


In [10]:
train_generator.class_indices

{'Ext': 0, 'Ext_Int': 1, 'Int': 2, 'Meal': 3}

In [77]:
test_generator = train_datagen.flow_from_directory(new_paths[1],
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 751 images belonging to 4 classes.


# Transfert Learning

In [15]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output


x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# dernière couche que sert a prédire la bonne classe
predictions = Dense(nb_types, activation = 'softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [16]:
step_size_train=train_generator.n//train_generator.batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=1, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5,
                   validation_data = test_generator,
                   callbacks=[reduce_lr])

Epoch 1/5
30/30 [==============================] - 319s 11s/step - loss: 0.3008 - accuracy: 0.8998 - val_loss: 0.0000e+00 - val_accuracy: 0.9559
Epoch 2/5
30/30 [==============================] - 323s 11s/step - loss: 0.0499 - accuracy: 0.9875 - val_loss: 1.7583e-05 - val_accuracy: 0.9648
Epoch 3/5
30/30 [==============================] - 317s 11s/step - loss: 0.0187 - accuracy: 0.9945 - val_loss: 1.8676e-06 - val_accuracy: 0.9626

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/5
30/30 [==============================] - 324s 11s/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 1.3869 - val_accuracy: 0.9648

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 5/5
30/30 [==============================] - 327s 11s/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0250 - val_accuracy: 0.9648

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.


In [ ]:
# Save model and labels
os.makedirs(project_path + 'Models/' + model_name + '/', exist_ok=True)
save_model_classes(project_path + 'Models/',
                   model_name, train_generator, model)